In [2]:
import pandas as pd
import re
import numpy as np
import os

In [3]:
def _load_app_store(data_dir = './dataset.csv'):
    df = pd.read_csv(data_dir)
    
    df_en = df[(df.lang == 'en') & (df.source == 'app_review') & ~df['text'].isna()]
    
    X = df_en['text'].to_list()
    y = df_en['category'].map({'inq': 0, 'pbr': 1, 'irr': 2}).to_list()
    
    return X, y

In [4]:
X, y = _load_app_store()

In [1]:
## IMDB

In [11]:
import pandas as pd
import re
import numpy as np

In [12]:
import os

def _load_imdb(data_dir = '/home/jovyan/Active Learning/data/aclImdb/'):
    X = []
    y = []
    for partition in ["train", "test"]:
        for category  in ["pos", "neg"]:
            lable = 0 if category  == "neg" else 1

            path = os.path.join(data_dir, partition, category )
            files = os.listdir(path)
            for f_name in files:
                with open(os.path.join(path, f_name), "r") as f:
                    review = f.read()
                    X.append(review)
                    y.append(lable)

    return X, y

X, y = _load_imdb()

In [13]:
# to remove URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# to remove html tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [14]:
df = pd.DataFrame(data=(X))
df.head()

,0
0,Although I have not seen this mini-series in o...
1,In terms of visual beauty this movie is outsta...
2,"Meatball Machine is an amazing splatter film, ..."
3,I have watched this movie three times. The las...
4,"Okay, so the first few seasons took a while to..."


In [15]:
df[0] = df[0].apply(remove_URL)
df[0] = df[0].apply(remove_html)

In [16]:
np.save('imdb_X', df[0].tolist())

In [17]:
np.save('imdb_y', y)

In [19]:
import os.path
os.path.isfile('./imdb_X.npy') 

True